In [1]:
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

df = pd.read_csv('data\monthly_churn_with_econV2.csv', index_col=0)
df.dropna(axis='index',inplace=True)

In [2]:
df

irrel_feat = ['DATE', 'customer_id','creation_date']

hi_corr = []

# feature creation

'''
- Account age (keep existing and split into)                                -COMPL
- Re-generate dataset with DoB                                              -COMPL
- with/deposit counts for previous 3 months                                 -COMPL  
- aggregate count/sum of transactions over past 3 month period              -COMPL
- avg count/sum over total life of the account                              -COMPL
- absolute count/sum over life of the account                               -COMPL              
'''

# Model Selection
''' 
- Regression models:
    3month churn definition
    6month churn definition

- Classification models:
    3month churn definition

- Time Series:
    time series validation model

'''


# Churn Definition
'''
- 0 balance remaining
- dates since last deposit greater than some threshold
- dates since inactivity
- financial % churn                                                         -SELECTED
- clustetred based churn definition                                         -IRRELEVENT
'''

#Scrum meeting
'''


'''
df

,DATE,UNEMP,SENT,PSR,GDP/C,EXP/C,customer_id,creation_date,start_balance,state,...,monthly_transactions,no_of_monthly_withdrawals,no_of_monthly_deposits,dob,churn,total_cum_sum_withdrawal,total_cum_sum_deposit,end_of_month_balance,total_cum_count_transactions,total_cum_count_interactions
0,2007-01,4.6,96.9,3.2,51492.0,31842.000000,91,2007-01-31,10180.56,California,...,2,1,1,1993-07-01,0,-5295.18,3034.26,7919.64,0,0
1,2007-02,4.5,91.3,3.6,51563.0,31936.333333,91,2007-01-31,10180.56,California,...,0,0,0,1993-07-01,0,-5295.18,3034.26,7919.64,1,1
2,2007-03,4.4,88.4,3.9,51634.0,32030.666667,91,2007-01-31,10180.56,California,...,0,0,0,1993-07-01,0,-5295.18,3034.26,7919.64,2,2
3,2007-01,4.6,96.9,3.2,51492.0,31842.000000,92,2007-01-31,4757.68,New York,...,0,0,0,1985-12-05,0,0.00,0.00,4757.68,0,0
4,2007-02,4.5,91.3,3.6,51563.0,31936.333333,92,2007-01-31,4757.68,New York,...,1,0,1,1985-12-05,0,0.00,1164.90,5922.58,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128883,2020-05,13.2,72.3,24.9,52393.0,39342.000000,116102,2020-05-31,3110.96,Georgia,...,2,1,1,1969-03-02,0,-1223.94,872.99,2760.01,0,0
2128884,2020-05,13.2,72.3,24.9,52393.0,39342.000000,116103,2020-05-31,12063.92,New York,...,3,2,1,1966-10-25,0,-4357.62,3354.40,11060.70,0,0
2128885,2020-05,13.2,72.3,24.9,52393.0,39342.000000,116104,2020-05-31,11011.36,Oklahoma,...,2,1,1,1986-04-30,0,-4002.30,2900.20,9909.26,0,0
2128886,2020-05,13.2,72.3,24.9,52393.0,39342.000000,116105,2020-05-31,8861.08,New Mexico,...,1,0,1,1983-08-25,0,0.00,2246.93,11108.01,0,0


In [3]:
df['max_account'] = df.groupby('customer_id')['end_of_month_balance'].transform(max)
df['net_diff'] = df['monthly_deposit'] + df['monthly_withdrawal'] 

df['start_month_balance'] = df['end_of_month_balance'] - df['net_diff']

df['3month_rolling_sum'] = df.groupby('customer_id')['net_diff'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
df['6month_rolling_sum'] = df.groupby('customer_id')['net_diff'].rolling(window=6, min_periods=1).sum().reset_index(level=0, drop=True)
df['12month_rolling_sum'] = df.groupby('customer_id')['net_diff'].rolling(window=12, min_periods=1).sum().reset_index(level=0, drop=True)

df['3month_rolling_max'] = df.groupby('customer_id')['start_month_balance'].rolling(window=3, min_periods=1).max().reset_index(level=0, drop=True)
df['6month_rolling_max'] = df.groupby('customer_id')['start_month_balance'].rolling(window=6, min_periods=1).max().reset_index(level=0, drop=True)
df['12month_rolling_max'] = df.groupby('customer_id')['start_month_balance'].rolling(window=12, min_periods=1).max().reset_index(level=0, drop=True)

df['three_monthly_decay'] = df.apply(lambda x: x['3month_rolling_sum']/x['3month_rolling_max'] if (x['3month_rolling_max'] >= 10) & (x['3month_rolling_sum'] != 0) else 0, axis=1)
df['6_monthly_decay'] = df.apply(lambda x: x['6month_rolling_sum']/x['6month_rolling_max'] if (x['6month_rolling_max'] >= 10) & (x['6month_rolling_sum'] != 0) else 0, axis=1)
df['12_monthly_decay'] = df.apply(lambda x: x['12month_rolling_sum']/x['12month_rolling_max'] if (x['12month_rolling_max'] >= 10) & (x['12month_rolling_sum'] != 0) else 0, axis=1)

df['three_monthly_decay'] = df.apply(lambda x : x['three_monthly_decay'] if x['three_monthly_decay'] < 3 else 3,axis=1)
df['6_monthly_decay'] = df.apply(lambda x : x['6_monthly_decay'] if x['6_monthly_decay'] < 3 else 3,axis=1)
df['12_monthly_decay'] = df.apply(lambda x : x['12_monthly_decay'] if x['12_monthly_decay'] < 3 else 3,axis=1)

df['UNEMP_rolling'] = df.groupby('customer_id')['UNEMP'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)


In [4]:
df['churn0.1'] = df.apply(lambda x: 1 if (x.three_monthly_decay < -0.1) | (x.churn == 1) else 0, axis=1)
df['churn0.3'] = df.apply(lambda x: 1 if (x.three_monthly_decay < -0.3) | (x.churn == 1) else 0, axis=1)
df['churn0.5'] = df.apply(lambda x: 1 if (x.three_monthly_decay < -0.5) | (x.churn == 1) else 0, axis=1)
df['churn0.7'] = df.apply(lambda x: 1 if (x.three_monthly_decay < -0.7) | (x.churn == 1) else 0, axis=1)
df['churn0.9'] = df.apply(lambda x: 1 if (x.three_monthly_decay < -0.9) | (x.churn == 1) else 0, axis=1)

'''
print('0.1',df['churn0.1'].sum())
print('0.3',df['churn0.3'].sum())
print('0.5',df['churn0.5'].sum())
print('0.7',df['churn0.7'].sum())
print('0.9',df['churn0.9'].sum()) 
print('0.1 \n -----------------\n', df.groupby('customer_id')['churn0.1'].sum().value_counts())
print('0.3 \n -----------------\n', df.groupby('customer_id')['churn0.3'].sum().value_counts())
print('0.5 \n -----------------\n', df.groupby('customer_id')['churn0.5'].sum().value_counts())
print('0.7 \n -----------------\n', df.groupby('customer_id')['churn0.7'].sum().value_counts())
print('0.9 \n -----------------\n', df.groupby('customer_id')['churn0.9'].sum().value_counts())
'''


"\nprint('0.1',df['churn0.1'].sum())\nprint('0.3',df['churn0.3'].sum())\nprint('0.5',df['churn0.5'].sum())\nprint('0.7',df['churn0.7'].sum())\nprint('0.9',df['churn0.9'].sum()) \nprint('0.1 \n -----------------\n', df.groupby('customer_id')['churn0.1'].sum().value_counts())\nprint('0.3 \n -----------------\n', df.groupby('customer_id')['churn0.3'].sum().value_counts())\nprint('0.5 \n -----------------\n', df.groupby('customer_id')['churn0.5'].sum().value_counts())\nprint('0.7 \n -----------------\n', df.groupby('customer_id')['churn0.7'].sum().value_counts())\nprint('0.9 \n -----------------\n', df.groupby('customer_id')['churn0.9'].sum().value_counts())\n"

In [6]:

df['CHURN'] = df['churn0.5']

df.drop(['churn0.1','churn0.3','churn0.5','churn0.7','churn0.9'],axis=1)

grouped = df.groupby('customer_id')

df['CHURN'] = grouped['churn0.5'].shift(periods=-1,axis=0)
df['CHURN'] = df['CHURN'].fillna(0)

In [8]:
df['customer_id'].value_counts().head(50)


df2 = df.drop(['UNEMP','SENT','state','PSR','GDP/C','EXP/C','monthly_interactions','monthly_transactions','no_of_monthly_withdrawals','dob','churn','no_of_monthly_deposits','total_cum_sum_deposit'],axis=1)

df['age'] = (pd.to_datetime(df['DATE']) - pd.to_datetime(df['dob'])).astype('timedelta64[Y]')

In [14]:
df.head(31)

,DATE,UNEMP,SENT,PSR,GDP/C,EXP/C,customer_id,creation_date,start_balance,state,...,6_monthly_decay,12_monthly_decay,UNEMP_rolling,churn0.1,churn0.3,churn0.5,churn0.7,churn0.9,CHURN,age
0,2007-01,4.6,96.9,3.2,51492.000000,31842.000000,91,2007-01-31,10180.56,California,...,-0.222082,-0.222082,4.600000,1,0,0,0,0,0.0,13.0
1,2007-02,4.5,91.3,3.6,51563.000000,31936.333333,91,2007-01-31,10180.56,California,...,-0.222082,-0.222082,4.550000,1,0,0,0,0,0.0,13.0
2,2007-03,4.4,88.4,3.9,51634.000000,32030.666667,91,2007-01-31,10180.56,California,...,-0.222082,-0.222082,4.500000,1,0,0,0,0,0.0,13.0
3,2007-01,4.6,96.9,3.2,51492.000000,31842.000000,92,2007-01-31,4757.68,New York,...,0.000000,0.000000,4.600000,0,0,0,0,0,0.0,21.0
4,2007-02,4.5,91.3,3.6,51563.000000,31936.333333,92,2007-01-31,4757.68,New York,...,0.244846,0.244846,4.550000,0,0,0,0,0,0.0,21.0
5,2007-03,4.4,88.4,3.9,51634.000000,32030.666667,92,2007-01-31,4757.68,New York,...,0.408991,0.408991,4.500000,0,0,0,0,0,0.0,21.0
6,2007-04,4.5,87.1,3.8,51705.000000,32125.000000,92,2007-01-31,4757.68,New York,...,0.523736,0.523736,4.466667,0,0,0,0,0,0.0,21.0
7,2007-05,4.4,88.3,3.6,51764.000000,32229.333333,92,2007-01-31,4757.68,New York,...,0.357934,0.357934,4.433333,0,0,0,0,0,0.0,21.0
8,2007-06,4.6,85.3,3.5,51823.000000,32333.666667,92,2007-01-31,4757.68,New York,...,0.357934,0.357934,4.500000,0,0,0,0,0,0.0,21.0
9,2007-07,4.7,90.4,3.4,51882.000000,32438.000000,92,2007-01-31,4757.68,New York,...,0.533356,0.533356,4.566667,0,0,0,0,0,0.0,21.0


In [15]:

df.to_csv('CHURN_dataset_with_AGE.csv')
#df.head(50)